In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

/home/tunguz/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.6.0rc1'

In [3]:
train_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-sep-2021/xgtrain.csv')
test_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-sep-2021/xgval.csv')

In [4]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [5]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-a19c00c3-2832-fe38-1c43-c18db3e909da",
                                                 "GPU-58b97c92-e879-49d3-85b5-1d9615f10873",
                                                 "GPU-d21cfed4-2e1a-f313-839c-ea008aca027a",
                                                 "GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac"])
client = Client(cluster)

2022-05-06 12:54:25,562 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-06 12:54:25,619 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-06 12:54:25,619 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-06 12:54:25,640 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [6]:
train_x.head()

,f1,f10,f100,f101,f102,f103,f104,f105,f106,f107,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,0.872615,-0.147912,0.315128,0.557139,-0.241808,-0.151454,0.890242,0.710130,1.357901,0.166360,...,0.442751,-2.865241,1.854788,-0.634225,0.167534,1.108749,-0.005678,-0.173913,-0.276507,-0.001903
1,-0.220001,-0.225827,-0.102546,0.390172,0.266328,0.271725,0.254364,-0.748292,0.166147,-0.161530,...,0.138979,-0.807670,1.880417,0.103766,0.216737,-0.208940,0.807075,0.017391,-0.281889,-0.483251
2,0.874560,-0.031575,-1.065833,0.082266,-0.404141,-0.124296,-0.210196,-0.144495,-0.259467,0.504897,...,0.092490,0.427859,0.201527,-0.353750,0.658368,0.631307,-0.197037,-0.460870,1.096485,-0.106585
3,-0.586468,0.569328,0.795074,-0.150262,1.174618,-0.068845,-0.256327,0.746887,1.405473,0.617058,...,-0.410961,0.173439,2.067397,0.082642,1.151074,-0.218919,-0.249827,-0.582609,-0.255763,0.917016
4,-0.704799,-0.169111,0.223030,-0.409437,1.530684,0.106552,0.248217,NaN,2.499167,-0.405990,...,-0.574557,-1.221557,5.860248,1.387319,-0.205065,1.830290,-0.072589,1.147826,0.180360,4.334793


In [7]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)
num_round = 1000

In [8]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [9]:
study = optuna.create_study(direction='maximize')

[I 2022-05-06 13:03:33,733] A new study created in memory with name: no-name-d2c31bb4-57bb-4a9c-b3ea-f62f455e0615


In [10]:
%%time
study.optimize(objective, n_trials=2)

[13:03:47] task [xgboost.dask]:tcp://127.0.0.1:33229 got new rank 0
[13:03:47] task [xgboost.dask]:tcp://127.0.0.1:42061 got new rank 1
[13:03:47] task [xgboost.dask]:tcp://127.0.0.1:35597 got new rank 2
[13:03:47] task [xgboost.dask]:tcp://127.0.0.1:35527 got new rank 3
[I 2022-05-06 13:04:05,012] Trial 0 finished with value: 0.7985885603265515 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.00473394785130864, 'alpha': 0.013282211872668299, 'colsample_bytree': 0.4880556065992996, 'subsample': 0.4763951436523317, 'learning_rate': 0.008626717298421183, 'max_depth': 13, 'min_child_weight': 185, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7985885603265515.
[13:04:05] task [xgboost.dask]:tcp://127.0.0.1:42061 got new rank 0
[13:04:05] task [xgboost.dask]:tcp://127.0.0.1:35527 got new rank 1
[13:04:05] task [xgboost.dask]:tcp://127.0.0.1:33229 got new rank 2
[13:04:05] task [xgboost.dask]:tcp://127.0.0.1:35597 got new rank 3
[I 2022-05-06

CPU times: user 13.7 s, sys: 7.85 s, total: 21.5 s
Wall time: 2min 3s


In [11]:
%%time
study.optimize(objective, n_trials=5)

[13:05:50] task [xgboost.dask]:tcp://127.0.0.1:42061 got new rank 0
[13:05:50] task [xgboost.dask]:tcp://127.0.0.1:35597 got new rank 1
[13:05:50] task [xgboost.dask]:tcp://127.0.0.1:35527 got new rank 2
[13:05:50] task [xgboost.dask]:tcp://127.0.0.1:33229 got new rank 3
[I 2022-05-06 13:06:04,907] Trial 2 finished with value: 0.8004811474165013 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.5327228319645178, 'alpha': 0.35458379690653347, 'colsample_bytree': 0.8355933673686449, 'subsample': 0.521253313155127, 'learning_rate': 0.01648251416545129, 'max_depth': 9, 'min_child_weight': 193, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.8004811474165013.
[13:06:04] task [xgboost.dask]:tcp://127.0.0.1:35527 got new rank 0
[13:06:04] task [xgboost.dask]:tcp://127.0.0.1:42061 got new rank 1
[13:06:04] task [xgboost.dask]:tcp://127.0.0.1:35597 got new rank 2
[13:06:04] task [xgboost.dask]:tcp://127.0.0.1:33229 got new rank 3
[I 2022-05-06 13:0

CPU times: user 12.3 s, sys: 2.75 s, total: 15.1 s
Wall time: 1min 55s


/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 1097951 should have a CUDA context assigned to device GPU-a19c00c3-2832-fe38-1c43-c18db3e909da, but instead the CUDA context is on device 0. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 1097954 should have a CUDA context assigned to device GPU-58b97c92-e879-49d3-85b5-1d9615f10873, but instead the CUDA context is on device 1. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warning